<a href="https://colab.research.google.com/github/Khashayarvani/MSIT3103-GenAI-assignments/blob/main/Khashayar_Vani_Google_Colab_assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## Dataset selection

This time I chose **CNN/DailyMail** Dataset and I ran it in Google Colab - First I install Evaluate then ran first part of codes.



In [3]:

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForMaskedLM,
    AutoModelForSeq2SeqLM,
    DataCollatorForLanguageModeling,
    DataCollatorForSeq2Seq,
    Trainer,
    TrainingArguments,
)
import evaluate
from transformers import logging

# Silence warnings for cleaner output
logging.set_verbosity_error()

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [2]:
%pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.3 MB/s eta 0:00:00



I changed the scale of Dataset, Train size from 500 to 700 and val size from 100 to 300.




In [4]:

# Load the cnn_dailymail dataset (version 3.0.0)
dataset = load_dataset("cnn_dailymail", "3.0.0")

# For quick experimentation, take a small subset
train_size = 700
val_size = 300
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(train_size))
small_val_dataset = dataset["validation"].shuffle(seed=42).select(range(val_size))

print("Dataset splits:", dataset.keys())
print("Example training record:", small_train_dataset[0])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset splits: dict_keys(['train', 'validation', 'test'])
Example training record: {'article': "By . Anthony Bond . PUBLISHED: . 07:03 EST, 2 March 2013 . | . UPDATED: . 08:07 EST, 2 March 2013 . Three members of the same family who died in a static caravan from carbon monoxide poisoning would have been unconscious 'within minutes', investigators said today. The bodies of married couple John and Audrey Cook were discovered alongside their daughter, Maureen, at the mobile home they shared on Tremarle Home Park in Camborne, west Cornwall. The inquests have now opened into the deaths last Saturday, with investigators saying the three died along with the family's pet dog, of carbon monoxide poisoning from a cooker. Tragic: The inquests have opened into the deaths of three members of the same family who were found in their static caravan last weekend. John and Audrey Cook are pictured . Awful: The family died following carbon monoxide poisoning at this caravan at the Tremarle Home Park in 


## Decoder‑only model: GPT‑2 fine‑tuning

I changed it from -100 to -300


In [5]:
# Load tokenizer and model for GPT-2
gpt2_model_name = "gpt2"
gpt2_tokenizer = AutoTokenizer.from_pretrained(gpt2_model_name)

# Add a padding token (GPT-2 does not have one)
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

# Define the preprocessing function for GPT-2
def preprocess_gpt2(examples):
    prefix = "summarize: "
    inputs = [prefix + art for art in examples["article"]]
    targets = examples["highlights"]

    # Tokenize inputs and targets together, pad to max_length
    model_inputs = gpt2_tokenizer(
        inputs, text_target=targets,
        max_length=512, truncation=True, padding="max_length"
    )

    # Ensure labels are padded to max_length as well
    if "labels" in model_inputs:
        labels = model_inputs["labels"]
        for i in range(len(labels)):
            labels[i] = labels[i] + [-300] * (512 - len(labels[i])) if len(labels[i]) < 512 else labels[i][:512]
        model_inputs["labels"] = labels
    return model_inputs

# Apply preprocessing to the small datasets
train_gpt2 = small_train_dataset.map(preprocess_gpt2, batched=True, remove_columns=dataset["train"].column_names)
val_gpt2 = small_val_dataset.map(preprocess_gpt2, batched=True, remove_columns=dataset["validation"].column_names)

print("Sample tokenized GPT-2 input:")
print(gpt2_tokenizer.decode(train_gpt2[0]["input_ids"][:100]))


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Sample tokenized GPT-2 input:
summarize: By . Anthony Bond . PUBLISHED: . 07:03 EST, 2 March 2013 . | . UPDATED: . 08:07 EST, 2 March 2013 . Three members of the same family who died in a static caravan from carbon monoxide poisoning would have been unconscious 'within minutes', investigators said today. The bodies of married couple John and Audrey Cook were discovered alongside their daughter, Maureen, at the mobile home they shared on Tremarle Home Park in Cam


In [9]:
print("Input IDs and Labels dimensions for first 13 samples:")
for i in range(13):
    print(f"Sample {i}: Input IDs length = {len(train_gpt2[i]['input_ids'])}, Labels length = {len(train_gpt2[i]['labels'])}")

Input IDs and Labels dimensions for first 13 samples:
Sample 0: Input IDs length = 512, Labels length = 512
Sample 1: Input IDs length = 512, Labels length = 512
Sample 2: Input IDs length = 512, Labels length = 512
Sample 3: Input IDs length = 512, Labels length = 512
Sample 4: Input IDs length = 512, Labels length = 512
Sample 5: Input IDs length = 512, Labels length = 512
Sample 6: Input IDs length = 512, Labels length = 512
Sample 7: Input IDs length = 512, Labels length = 512
Sample 8: Input IDs length = 512, Labels length = 512
Sample 9: Input IDs length = 512, Labels length = 512
Sample 10: Input IDs length = 512, Labels length = 512
Sample 11: Input IDs length = 512, Labels length = 512
Sample 12: Input IDs length = 512, Labels length = 512



### GPT‑2 training configuration

We use the Hugging Face `Trainer` API to fine‑tune the GPT‑2 model.  A `DataCollatorForLanguageModeling` automatically pads the inputs and labels and performs dynamic masking where appropriate (although in our custom loss masking we already set `-100` values).  The training arguments below specify a small number of epochs and batch sizes for illustration; adjust these for a full training run.


In [10]:

# Define data collator
data_collator_gpt2 = DataCollatorForLanguageModeling(tokenizer=gpt2_tokenizer, mlm=False)

# Load the GPT-2 model
gpt2_model = AutoModelForCausalLM.from_pretrained(gpt2_model_name)

# Training arguments
training_args_gpt2 = TrainingArguments(
    output_dir="./gpt2-summarization",
    eval_strategy="steps",
    eval_steps=300,
    logging_steps=300,
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    num_train_epochs=3,
    weight_decay=0.03,
    save_steps=700,
    save_total_limit=2,
    warmup_steps=70,
    gradient_accumulation_steps=7,
    fp16=torch.cuda.is_available(),
    report_to=[],  # disable logging to wandb
)

# Create Trainer for GPT-2
trainer_gpt2 = Trainer(
    model=gpt2_model,
    args=training_args_gpt2,
    train_dataset=train_gpt2,
    eval_dataset=val_gpt2,
    data_collator=data_collator_gpt2,
)

# Uncomment the line below to train; training can take several minutes even on small subsets
trainer_gpt2.train()


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

{'train_runtime': 153.353, 'train_samples_per_second': 13.694, 'train_steps_per_second': 0.665, 'train_loss': 3.128665699678309, 'epoch': 3.0}


TrainOutput(global_step=102, training_loss=3.128665699678309, metrics={'train_runtime': 153.353, 'train_samples_per_second': 13.694, 'train_steps_per_second': 0.665, 'train_loss': 3.128665699678309, 'epoch': 3.0})

In [ ]:
# Test the model with a sample input
sample_input = "summarize: The quick brown fox jumps over the lazy dog."
inputs = gpt2_tokenizer(sample_input, return_tensors="pt").to(device)
outputs = gpt2_model.generate(inputs["input_ids"], max_length=50, num_return_sequences=1)
print("Generated summary:", gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True))

Generated summary: summarize: The quick brown fox jumps over the lazy dog. The dog is a little bit of a mystery to the dog owner, but the dog's owner is a big fan of the dog. The dog is a little bit of a mystery



## Encoder‑only model: BERT fine‑tuning



In [11]:
# Print first 10 samples of the dataset
print("First 10 samples of the dataset:")
print("small_train_dataset[0]:", small_train_dataset[0].keys())
for i in range(10):
    print(f"\t\tsmall_train_dataset[{i}]: {small_train_dataset[i]['article'][:350]}... \n\t->\n\t\t{small_train_dataset[i]['highlights']}")

First 10 samples of the dataset:
small_train_dataset[0]: dict_keys(['article', 'highlights', 'id'])
		small_train_dataset[0]: By . Anthony Bond . PUBLISHED: . 07:03 EST, 2 March 2013 . | . UPDATED: . 08:07 EST, 2 March 2013 . Three members of the same family who died in a static caravan from carbon monoxide poisoning would have been unconscious 'within minutes', investigators said today. The bodies of married couple John and Audrey Cook were discovered alongside their dau... 
	->
		John and .
Audrey Cook were discovered alongside their daughter, Maureen .
They were found at Tremarle Home Park in Cornwall .
Investigators say the three died of carbon monoxide .
poisoning .
		small_train_dataset[1]: UNITED NATIONS (CNN) -- A rare meeting of U.N. Security Council heads of state, led for the first time by a U.S. president, adopted a resolution focused on stopping the spread of nuclear weapons Thursday. President Obama is the first U.S. leader to head a United Nations Security Council meetin

In [12]:

# Load tokenizer and model for BERT
bert_model_name = "bert-base-uncased"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

# Define preprocessing: combine article and summary

def preprocess_bert(examples):
    concatenated_texts = [
        art + " " + summ
        for art, summ in zip(examples["article"], examples["highlights"])
    ]
    model_inputs = bert_tokenizer(concatenated_texts, max_length=512, truncation=True)
    return model_inputs

train_bert = small_train_dataset.map(preprocess_bert, batched=True, remove_columns=dataset["train"].column_names)
val_bert = small_val_dataset.map(preprocess_bert, batched=True, remove_columns=dataset["validation"].column_names)

# Data collator with MLM
mlm_probability = 0.15
data_collator_bert = DataCollatorForLanguageModeling(tokenizer=bert_tokenizer, mlm=True, mlm_probability=mlm_probability)

# Load BERT model
bert_model = AutoModelForMaskedLM.from_pretrained(bert_model_name)

# Training arguments for BERT
training_args_bert = TrainingArguments(
    output_dir="./bert-mlm",
    eval_strategy="steps",
    eval_steps=50,
    logging_steps=50,
    per_device_train_batch_size=7,
    per_device_eval_batch_size=7,
    num_train_epochs=1,
    weight_decay=0.01,
    save_steps=700,
    save_total_limit=1,
    warmup_steps=70,
    gradient_accumulation_steps=1,
    fp16=torch.cuda.is_available(),
    report_to=[],
)

trainer_bert = Trainer(
    model=bert_model,
    args=training_args_bert,
    train_dataset=train_bert,
    eval_dataset=val_bert,
    data_collator=data_collator_bert,
)

# Uncomment the line below to train the BERT model
trainer_bert.train()
# save the model
bert_model.save_pretrained("./bert-mlm")
bert_tokenizer.save_pretrained("./bert-mlm")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

{'loss': 2.3564, 'grad_norm': 6.487233638763428, 'learning_rate': 3.5e-05, 'epoch': 0.5}
{'eval_loss': 1.911010980606079, 'eval_runtime': 3.4988, 'eval_samples_per_second': 85.744, 'eval_steps_per_second': 12.29, 'epoch': 0.5}
{'loss': 2.0993, 'grad_norm': 6.174864292144775, 'learning_rate': 1.6666666666666667e-06, 'epoch': 1.0}
{'eval_loss': 1.8185157775878906, 'eval_runtime': 3.6695, 'eval_samples_per_second': 81.754, 'eval_steps_per_second': 11.718, 'epoch': 1.0}
{'train_runtime': 49.2903, 'train_samples_per_second': 14.202, 'train_steps_per_second': 2.029, 'train_loss': 2.22784309387207, 'epoch': 1.0}


('./bert-mlm/tokenizer_config.json',
 './bert-mlm/special_tokens_map.json',
 './bert-mlm/vocab.txt',
 './bert-mlm/added_tokens.json',
 './bert-mlm/tokenizer.json')

In [13]:
# Print output of bert model for sample input
sample_input = "The quick brown fox jumps over the lazy dog."
inputs = bert_tokenizer(sample_input, return_tensors="pt").to(device)
outputs = bert_model(**inputs)
print(f"Sample {i}: {outputs.logits.shape} logits")

# Freeze the BERT model and train a classifier on top
# load a news sentiment classification dataset
from datasets import load_dataset
news_dataset = load_dataset("ag_news", split="train[:1000]")
print("News dataset loaded with", len(news_dataset), "samples.")
# Preprocess the dataset for BERT
def preprocess_news(examples):
    model_inputs = bert_tokenizer(examples["text"], max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = examples["label"]
    return model_inputs
train_news = news_dataset.map(preprocess_news, batched=True, remove_columns=news_dataset.column_names)
# Use the trained model as a feature extractor and add a classification head
def get_news_classifier_model():
    """
    load the trained BERT model and add a classification head
    """
    model = AutoModelForMaskedLM.from_pretrained("./bert-mlm") # Load the trained BERT model
    # Freeze the BERT model
    for param in model.bert.parameters():
        param.requires_grad = False
    # Add a classification head
    model.classifier = torch.nn.Linear(model.config.hidden_size, 4)  # 4 classes for AG News
    return model

news_classifier_model = get_news_classifier_model()
# Training arguments for news classifier
news_training_args = TrainingArguments(
    output_dir="./news-classifier",
    eval_strategy="steps",
    eval_steps=300,
    logging_steps=300,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    num_train_epochs=6,
    weight_decay=0.03,
    save_steps=700,
    save_total_limit=1,
    warmup_steps=70,
    gradient_accumulation_steps=1,
    fp16=torch.cuda.is_available(),
    report_to=[],
)
# Create Trainer for news classifier
news_trainer = Trainer(
    model=news_classifier_model,
    args=news_training_args,
    train_dataset=train_news,
    eval_dataset=train_news,  # For simplicity, using the same dataset for eval
    data_collator=DataCollatorForLanguageModeling(tokenizer=bert_tokenizer, mlm=False),
)
# Uncomment the line below to train the news classifier
news_trainer.train()

Sample 9: torch.Size([1, 12, 30522]) logits


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

News dataset loaded with 1000 samples.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'loss': 0.3772, 'grad_norm': 0.39072564244270325, 'learning_rate': 2.361751152073733e-05, 'epoch': 3.571428571428571}
{'eval_loss': 0.14793051779270172, 'eval_runtime': 10.4936, 'eval_samples_per_second': 95.296, 'eval_steps_per_second': 8.005, 'epoch': 3.571428571428571}
{'train_runtime': 95.7249, 'train_samples_per_second': 62.68, 'train_steps_per_second': 5.265, 'train_loss': 0.2980062772357275, 'epoch': 6.0}


TrainOutput(global_step=504, training_loss=0.2980062772357275, metrics={'train_runtime': 95.7249, 'train_samples_per_second': 62.68, 'train_steps_per_second': 5.265, 'train_loss': 0.2980062772357275, 'epoch': 6.0})

In [ ]:
# Print sample output of the news classifier
# The previous approach used AutoModelForMaskedLM, which is not suitable for classification.
# Instead, use AutoModelForSequenceClassification for news classification.
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)
import evaluate
bert_model_name = "bert-base-uncased"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load a BERT model with a classification head
news_classifier_model = AutoModelForSequenceClassification.from_pretrained(
    bert_model_name, num_labels=4
).to(device)

# Training arguments for news classifier remain the same
news_training_args = TrainingArguments(
    output_dir="./news-classifier",
    eval_strategy="steps",
    eval_steps=100,
    logging_steps=100,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_steps=500,
    save_total_limit=1,
    warmup_steps=50,
    gradient_accumulation_steps=1,
    fp16=torch.cuda.is_available(),
    report_to=[],
)

# Load the AG News dataset
from datasets import load_dataset
news_dataset = load_dataset("ag_news", split="train[:1000]")
print("News dataset loaded with", len(news_dataset), "samples.")

# Preprocess the dataset for BERT
def preprocess_news(examples):
    model_inputs = bert_tokenizer(examples["text"], max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = examples["label"]
    return model_inputs

train_news = news_dataset.map(preprocess_news, batched=True, remove_columns=news_dataset.column_names)

# Create Trainer for news classifier
news_trainer = Trainer(
    model=news_classifier_model,
    args=news_training_args,
    train_dataset=train_news,
    eval_dataset=train_news,  # For simplicity, using the same dataset for eval
    tokenizer=bert_tokenizer,
)
# Uncomment the line below to train the news classifier
news_trainer.train()

# Print sample output of the news classifier
sample_news_input = "Breaking news: The stock market crashes as investors panic."
inputs = bert_tokenizer(sample_news_input, return_tensors="pt").to(device)
with torch.no_grad():
    outputs = news_classifier_model(**inputs)
pred_class = torch.argmax(outputs.logits, dim=-1).item()
print(f"Sample news input: {sample_news_input}")
print(f"Predicted class: {pred_class}")
print("Available classes:", news_dataset.features["label"].names)

News dataset loaded with 1000 samples.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/tmp/ipython-input-2350930871.py:54: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  news_trainer = Trainer(


{'loss': 1.0767, 'grad_norm': 4.787654399871826, 'learning_rate': 4.831034482758621e-05, 'epoch': 0.2}
{'eval_loss': 0.5406842827796936, 'eval_runtime': 9.6448, 'eval_samples_per_second': 103.683, 'eval_steps_per_second': 51.842, 'epoch': 0.2}
{'loss': 0.6575, 'grad_norm': 0.037821199744939804, 'learning_rate': 4.486206896551725e-05, 'epoch': 0.4}
{'eval_loss': 0.7874549031257629, 'eval_runtime': 9.7327, 'eval_samples_per_second': 102.747, 'eval_steps_per_second': 51.373, 'epoch': 0.4}
{'loss': 0.7393, 'grad_norm': 23.732959747314453, 'learning_rate': 4.141379310344828e-05, 'epoch': 0.6}
{'eval_loss': 0.46051904559135437, 'eval_runtime': 9.521, 'eval_samples_per_second': 105.031, 'eval_steps_per_second': 52.516, 'epoch': 0.6}
{'loss': 0.6164, 'grad_norm': 0.08265659213066101, 'learning_rate': 3.796551724137931e-05, 'epoch': 0.8}
{'eval_loss': 0.4773515462875366, 'eval_runtime': 9.4362, 'eval_samples_per_second': 105.974, 'eval_steps_per_second': 52.987, 'epoch': 0.8}
{'loss': 0.5684, '

In [14]:
# Generate a confusion matrix
from sklearn.metrics import confusion_matrix
import numpy as np
# Get predictions for the entire training set
predictions = []
for i in range(0, len(train_news), 64):  # Batch size of 64
    batch = train_news[i:i+64]
    # batch is a dict with keys: input_ids, token_type_ids, attention_mask, labels
    # You need to decode input_ids to get the original text if you want to print or use text
    # For inference, use input_ids, attention_mask, etc.
    inputs = {
        "input_ids": torch.tensor(batch["input_ids"]).to(device),
        "attention_mask": torch.tensor(batch["attention_mask"]).to(device)
    }
    with torch.no_grad():
        outputs = news_classifier_model(**inputs)
    preds = torch.argmax(outputs.logits, dim=-1).cpu().numpy()
    predictions.extend(preds)
# Get true labels
true_labels = np.array(batch["labels"] for batch in train_news)  # This line should be fixed
# Instead, collect all labels from train_news:
true_labels = np.array(train_news["labels"])
# Generate confusion matrix
conf_matrix = confusion_matrix(true_labels, predictions)
print("Confusion Matrix:")
print(conf_matrix)
# Plot confusion matrix
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues",
            xticklabels=news_dataset.features["label"].names,
            yticklabels=news_dataset.features["label"].names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix for News Classifier")
plt.show()


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.73 GiB. GPU 0 has a total capacity of 14.74 GiB of which 3.70 GiB is free. Process 5769 has 11.04 GiB memory in use. Of the allocated memory 8.68 GiB is allocated by PyTorch, and 2.22 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


## Encoder‑decoder model: T5 fine‑tuning

T5 is a text‑to‑text model that uses a separate encoder and decoder.  It naturally handles generative tasks such as summarisation.  We prepend the prefix `"summarize: "` to each article, then tokenize the input and the summary separately.  A `DataCollatorForSeq2Seq` takes care of padding the inputs and shifting the decoder labels.  During evaluation we use greedy decoding to produce summaries and compute ROUGE scores against the reference summaries.


In [15]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Trainer, TrainingArguments
from datasets import load_dataset
# Load the cnn_dailymail dataset (version 3.0.0)
dataset = load_dataset("cnn_dailymail", "3.0.0")

# For quick experimentation, take a small subset
train_size = 500
val_size = 100
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(train_size))
small_val_dataset = dataset["validation"].shuffle(seed=42).select(range(val_size))
print("Dataset splits:", dataset.keys())
print("Example training record:", small_train_dataset[0])

# Load T5 tokenizer and model
t5_model_name = "t5-small"
t5_tokenizer = AutoTokenizer.from_pretrained(t5_model_name)

def preprocess_t5(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]
    model_inputs = t5_tokenizer(inputs, max_length=512, truncation=True)

    # Tokenize targets
    labels = t5_tokenizer(examples["highlights"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_t5 = small_train_dataset.map(preprocess_t5, batched=True, remove_columns=dataset["train"].column_names)
val_t5 = small_val_dataset.map(preprocess_t5, batched=True, remove_columns=dataset["validation"].column_names)

# Data collator for seq2seq tasks
data_collator_t5 = DataCollatorForSeq2Seq(tokenizer=t5_tokenizer, model=t5_model_name)

# Load T5 model
t5_model = AutoModelForSeq2SeqLM.from_pretrained(t5_model_name)

# Training arguments for T5
training_args_t5 = TrainingArguments(
    output_dir="./t5-summarization",
    eval_strategy="steps",
    eval_steps=300,
    logging_steps=300,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    save_steps=500,
    save_total_limit=1,
    warmup_steps=50,
    gradient_accumulation_steps=8,
    fp16=torch.cuda.is_available(),
    report_to=[],
)

trainer_t5 = Trainer(
    model=t5_model,
    args=training_args_t5,
    train_dataset=train_t5,
    eval_dataset=val_t5,
    data_collator=data_collator_t5,
    tokenizer=t5_tokenizer,
)

# Uncomment the line below to train the T5 model
trainer_t5.train()


Dataset splits: dict_keys(['train', 'validation', 'test'])
Example training record: {'article': "By . Anthony Bond . PUBLISHED: . 07:03 EST, 2 March 2013 . | . UPDATED: . 08:07 EST, 2 March 2013 . Three members of the same family who died in a static caravan from carbon monoxide poisoning would have been unconscious 'within minutes', investigators said today. The bodies of married couple John and Audrey Cook were discovered alongside their daughter, Maureen, at the mobile home they shared on Tremarle Home Park in Camborne, west Cornwall. The inquests have now opened into the deaths last Saturday, with investigators saying the three died along with the family's pet dog, of carbon monoxide poisoning from a cooker. Tragic: The inquests have opened into the deaths of three members of the same family who were found in their static caravan last weekend. John and Audrey Cook are pictured . Awful: The family died following carbon monoxide poisoning at this caravan at the Tremarle Home Park in 

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

...

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/tmp/ipython-input-4118490571.py:54: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_t5 = Trainer(


{'train_runtime': 15.7045, 'train_samples_per_second': 31.838, 'train_steps_per_second': 1.019, 'train_loss': 2.5961081981658936, 'epoch': 1.0}


TrainOutput(global_step=16, training_loss=2.5961081981658936, metrics={'train_runtime': 15.7045, 'train_samples_per_second': 31.838, 'train_steps_per_second': 1.019, 'train_loss': 2.5961081981658936, 'epoch': 1.0})

In [ ]:

# Define ROUGE metric
evaluate_rouge = evaluate.load("rouge")

def compute_metrics_rouge(preds, refs):
    # Compute ROUGE scores; use newline separation between sentences in each text
    result = evaluate_rouge.compute(predictions=preds, references=refs, use_stemmer=True)
    return {k: round(v * 100, 2) for k, v in result.items()}

# Function to generate summaries with GPT-2
def evaluate_gpt2(model, tokenizer, dataset, num_samples=10):
    model.eval()
    preds, refs = [], []
    for i, example in enumerate(dataset.select(range(num_samples))):
        prompt = "summarize: " + example["article"]
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(model.device)
        with torch.no_grad():
            output_ids = model.generate(**inputs, max_length=128)
        summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        preds.append(summary)
        refs.append(example["highlights"])
    rouge_scores = compute_metrics_rouge(preds, refs)
    return rouge_scores

# Function to compute perplexity from evaluation loss
def compute_perplexity(eval_output):
    loss = eval_output["eval_loss"]
    return round(torch.exp(torch.tensor(loss)).item(), 3)

# Evaluate GPT-2 (if trained) -- example usage
gpt2_eval_results = trainer_gpt2.evaluate()
gpt2_perplexity = compute_perplexity(gpt2_eval_results)
rouge_gpt2 = evaluate_gpt2(gpt2_model, gpt2_tokenizer, small_val_dataset)
print("GPT-2 Perplexity:", gpt2_perplexity)
print("GPT-2 ROUGE:", rouge_gpt2)

# BERT evaluation: compute F1 score - sklearn classification report
from sklearn.metrics import classification_report
def evaluate_bert(model, dataset):
    model.eval()
    predictions = []
    for i in range(0, len(dataset), 64):  # Batch size of 64
        batch = dataset[i:i+64]
        # batch is a dict with keys: input_ids, token_type_ids, attention_mask, labels
        # You need to decode input_ids to get the original text if you want to print or use text
        # For inference, use input_ids, attention_mask, etc.
        inputs = {
            "input_ids": torch.tensor(batch["input_ids"]).to(device),
            "attention_mask": torch.tensor(batch["attention_mask"]).to(device)
        }
        with torch.no_grad():
            outputs = news_classifier_model(**inputs)
        preds = torch.argmax(outputs.logits, dim=-1).cpu().numpy()
        predictions.extend(preds)

    true_labels = np.array(dataset["labels"])
    # Generate classification report
    report = classification_report(true_labels, predictions, output_dict=True)
    return report
# Evaluate on random 100 samples
bert_eval_results = evaluate_bert(news_classifier_model, train_news.shuffle(seed=42).select(range(100)))
import json
print("BERT Evaluation Report:", json.dumps(bert_eval_results, indent=2))

# Demonstrate fill-mask prediction with BERT
# from transformers import pipeline
# fill_mask = pipeline("fill-mask", model=bert_model, tokenizer=bert_tokenizer)
# sentence = "The weather today is [MASK]."
# print(fill_mask(sentence))

# T5 evaluation
# t5_eval_results = trainer_t5.evaluate()
# t5_perplexity = compute_perplexity(t5_eval_results)

# def evaluate_t5(model, tokenizer, dataset, num_samples=10):
#     model.eval()
#     preds, refs = [], []
#     for i, example in enumerate(dataset.select(range(num_samples))):
#         input_text = "summarize: " + example["article"]
#         inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512).to(model.device)
#         with torch.no_grad():
#             output_ids = model.generate(**inputs, max_length=128)
#         summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
#         preds.append(summary)
#         refs.append(example["highlights"])
#     rouge_scores = compute_metrics_rouge(preds, refs)
#     return rouge_scores

# rouge_t5 = evaluate_t5(t5_model, t5_tokenizer, small_val_dataset)
# print("T5 Perplexity:", t5_perplexity)
# print("T5 ROUGE:", rouge_t5)


ImportError: To be able to use evaluate-metric/rouge, you need to install the following dependencies['rouge_score'] using 'pip install rouge_score' for instance'